In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [17]:
df= pd.read_csv('/Users/chris/Library/Containers/com.tencent.xinWeChat/Data/Library/Application Support/com.tencent.xinWeChat/2.0b4.0.9/cb57adccddf5921e6681fe69a9347f09/Message/MessageTemp/79fbb2b8df6d7d1ae4bb6e6a26080f74/File/df_final.csv')

In [18]:
# List of columns to remove from the DataFrame
columns_to_remove = [
    'ADM_RATE_ALL',
    'CONTROL',
    'PCTFLOAN',
    'UGDS',
    'ICLEVEL',
    'CDR3',
    'COMP_ORIG_YR4_RT',
    'GRAD_DEBT_MDN',
    'TUITFTE'
]

# Remove the listed columns
df = df.drop(columns_to_remove, axis=1)

# Handle missing values
df = df.dropna()

# Split the data into features and target
X = df.drop('RPY_3YR_RT', axis=1)
y = df['RPY_3YR_RT']

# Split the data into a training set (80%) and a test set (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:


# Initialize lists to store accuracies for different numbers of principal components
train_accuracies = []
val_accuracies = []

In [25]:
# Convert the target variable into a binary classification problem using the median repayment rate as the threshold
threshold = df['RPY_3YR_RT'].median()
y_binary = (df['RPY_3YR_RT'] > threshold).astype(int)

# Split the data into training and testing sets with the binary target variable
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Standardize the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize lists to store accuracies for different numbers of principal components
val_accuracies = []

# Iterate over the possible number of components (1 to 7)
for n in range(1, 8):
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train_scaled)
    
    logistic_regression = LogisticRegression(max_iter=10000)
    logistic_regression.fit(X_train_pca, y_train)
    
    # Cross-validation
    val_accuracy = cross_val_score(logistic_regression, X_train_pca, y_train, cv=5, scoring='accuracy').mean()
    
    # Store the validation accuracy
    val_accuracies.append(val_accuracy)

# Determine the optimal number of components based on validation accuracy
optimal_n_components = val_accuracies.index(max(val_accuracies)) + 1

# Train the logistic regression model using the optimal number of components
pca = PCA(n_components=optimal_n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

logistic_regression = LogisticRegression(max_iter=10000)
logistic_regression.fit(X_train_pca, y_train)

# Evaluate the model's performance on training and test sets
y_train_pred = logistic_regression.predict(X_train_pca)
y_test_pred = logistic_regression.predict(X_test_pca)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

optimal_n_components, train_accuracy, test_accuracy


(7, 0.8590388095716871, 0.8432475884244373)